In [1437]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import log_loss,accuracy_score,classification_report,confusion_matrix
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold,KFold,GridSearchCV,GroupKFold,train_test_split,GroupKFold
from rfpimp import *
from xgboost import XGBClassifier
pd.set_option('display.max_rows',100)

In [1589]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')

In [1590]:
cab = {'B': 1, 'C': 2, 'A': 0, 'D':3 , 'E':4}
conf = {'B': 1, 'C': 2, 'A': 0}
dest = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'N': 12, 'M': 13}
gend = {'Male': 0, 'Female': 1}

In [1591]:
df_train['Type_of_Cab'] = df_train['Type_of_Cab'].map(cab)
df_train['Confidence_Life_Style_Index'] = df_train['Confidence_Life_Style_Index'].map(conf)
df_train['Destination_Type'] = df_train['Destination_Type'].map(dest)
df_train['Gender'] = df_train['Gender'].map(gend)

In [1592]:
df_train['Type_of_Cab'].fillna(-999,inplace=True)
df_train['Var1_isnull'] = df_train['Var1'].apply(lambda x: 1 if np.isnan(x) == True else 0)
df_train['Var1'].fillna(df_train['Var1'].median(),inplace=True)
df_train['Confidence_Life_Style_Index'].fillna(-999,inplace=True)
df_train['Life_Style_Index'].fillna(-999,inplace=True)

In [1593]:
df_train['Customer_isnull'] =  df_train['Customer_Since_Months'].apply(lambda x: 1 if np.isnan(x) == True else 0)
df_train['Customer_Since_Months'].fillna(df_train['Customer_Since_Months'].median(),inplace=True)

In [1594]:
df_train.drop('Trip_ID',axis=1,inplace=True)

In [1595]:
encoding = df_train.groupby('Type_of_Cab').size()
encoding = encoding/len(df_test)
df_train['enc_cab'] = df_train['Type_of_Cab'].map(encoding)

In [1596]:
# df_train.drop(['enc_cab'],axis=1,inplace=True)

In [1597]:
X_train = df_train.drop(['Surge_Pricing_Type'],axis=1)
y_train = df_train['Surge_Pricing_Type']

In [1598]:
# df_train = pd.get_dummies(data=df_train,columns=['Confidence_Life_Style_Index','Destination_Type'])

In [1599]:
# X_train,X_test,y_train,y_test = train_test_split(df_train.drop('Surge_Pricing_Type',axis=1),df_train['Surge_Pricing_Type'],test_size=0.3,stratify=df_train['Surge_Pricing_Type'],
#                                                  random_state=22)

In [1600]:
# X_test_final = X_test.copy()

In [1601]:
# x_x,X_valid,_,y_valid = train_test_split(X_test,y_test,test_size=0.25,stratify=y_test,random_state=22)

In [1602]:
random_states = [22,2019,2020,1997,17,1890,1980]

In [1660]:

m  = lgb.LGBMClassifier(random_state=22,n_jobs=-1,colsample_bytree=0.7,min_child_samples=25,num_leaves=47,n_estimators=120)
m.fit(X_train[k],y_train)
# print(accuracy_score(y_pred=m6.predict(X_valid[k]),y_true=y_valid))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=25, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=120, n_jobs=-1, num_leaves=47, objective=None,
               random_state=22, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [1614]:
print(confusion_matrix(y_pred=m.predict(X_valid[k]),y_true=y_valid))
print(classification_report(y_pred=m.predict(X_valid[k]),y_true=y_valid))

[[1167  651  223]
 [ 203 3443  609]
 [ 146  911 2522]]
              precision    recall  f1-score   support

           1       0.77      0.57      0.66      2041
           2       0.69      0.81      0.74      4255
           3       0.75      0.70      0.73      3579

    accuracy                           0.72      9875
   macro avg       0.74      0.70      0.71      9875
weighted avg       0.73      0.72      0.72      9875



In [1615]:
print(accuracy_score(y_pred=m.predict(X_test_final[k]),y_true=y_test))

0.7202967163725664


In [1548]:
X_train.shape

(131662, 15)

In [1555]:
k

['enc_cab',
 'Type_of_Cab',
 'Trip_Distance',
 'Cancellation_Last_1Month',
 'Confidence_Life_Style_Index',
 'Destination_Type',
 'Var3',
 'Customer_Rating',
 'Life_Style_Index',
 'Var1_isnull',
 'Var1',
 'Var2',
 'Customer_Since_Months',
 'Customer_isnull']

In [1674]:
gkf = list(StratifiedKFold(n_splits=5,shuffle=True,random_state=22).split(X_train,y_train))

In [1687]:
#Best Model
m  = lgb.LGBMClassifier(random_state=22,n_jobs=-1,colsample_bytree=1,min_child_samples=22,num_leaves=65,
                       lambda_l1=2.41,lambda_l2=5.21,learning_rate=0.05,n_estimators=220)

In [1688]:
params ={
    'n_estimators':range(210,230,2)
}
gsearch1 = GridSearchCV(estimator=m,param_grid=params,n_jobs=-1,iid=False,cv=gkf,scoring='accuracy',verbose=2)
gsearch1.fit(X_train[k],y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.3min finished


GridSearchCV(cv=[(array([     0,      1,      2, ..., 131658, 131659, 131660]),
                  array([     3,      5,      8, ..., 131652, 131653, 131661])),
                 (array([     0,      1,      2, ..., 131659, 131660, 131661]),
                  array([     6,     10,     13, ..., 131646, 131651, 131654])),
                 (array([     0,      2,      3, ..., 131659, 131660, 131661]),
                  array([     1,     37,     50, ..., 131656, 131657, 131658])),
                 (array([     0,      1,      3, ..., 131658, 131659, 131661]),
                  array([     2,      4,     15, ..., 131639, 131640, 131660])),...
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=220,
                                      n_jobs=-1, num_leaves=65, objective=None,
                                      random_state=22, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True

In [1689]:
gsearch1.best_params_,gsearch1.best_score_

({'n_estimators': 220}, 0.7048274907930706)

In [1197]:
'n_estimators':range(80,500,20),
'learning_rate':[0.1,0.005,0.05,0.2,0.15,0.3]
    'scale_pos_weight':[x/10 for x in range(5,21)]
    'num_leaves':range(10,20,1),
    'min_child_samples':range(40,60,1),
           'lambda_l1':[x/100 for x in range(0,20,1)],
    'lambda_l2':[x/100 for x in range(0,20,1)],

SyntaxError: invalid syntax (<ipython-input-1197-7e3749e85f55>, line 1)

In [1352]:
k

['Type_of_Cab',
 'Trip_Distance',
 'Cancellation_Last_1Month',
 'Confidence_Life_Style_Index',
 'Destination_Type',
 'Var3',
 'Customer_Rating',
 'Life_Style_Index',
 'Var1_isnull',
 'Var1',
 'Var2',
 'Customer_Since_Months',
 'Customer_isnull']

In [ ]:
random_states = [22,2019,2020,1997,17,1890,1980]

In [1708]:
cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=22)
results = pd.DataFrame(columns=['training_score', 'test_score'])
score_avg = []
for (train, test), i in zip(cv.split(X_train, y_train), range(5)):
#Best Model
    m1  =lgb.LGBMClassifier(random_state=22,n_jobs=-1,colsample_bytree=1,min_child_samples=22,num_leaves=65,
                       lambda_l1=2.41,lambda_l2=5.21,learning_rate=0.1,n_estimators=100)
    m1.fit(X_train[k].iloc[train], y_train.iloc[train])
    score_avg.append(accuracy_score(y_pred=m1.predict(X_train[k].iloc[test]),y_true=y_train.iloc[test]))
print(pd.DataFrame(data=score_avg))
print(sum(score_avg)/len(score_avg))

          0
0  0.704781
1  0.709452
2  0.705643
3  0.702833
4  0.704048
0.7053515552704576


In [1427]:
y_preds = np.mean([m1.predict(X_valid[k]),m2.predict(X_valid[k]),m4.predict(X_valid[k]),m3.predict(X_valid[k]),
                  m5.predict(X_valid[k]),m6.predict(X_valid[k]),m.predict(X_valid[k])],axis=0)

In [1430]:
y_preds = y_preds.round()

In [1431]:
print(accuracy_score(y_pred=y_preds,y_true=y_valid))

0.705620253164557


In [1653]:
print(confusion_matrix(y_pred=m.predict(X_valid[k]),y_true=y_valid))
print(classification_report(y_pred=m.predict(X_valid[k]),y_true=y_valid))

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.

In [1654]:
y_preds = np.mean([m1.predict(X_test_final[k]),m2.predict(X_test_final[k]),m4.predict(X_test_final[k]),m3.predict(X_test_final[k]),
                  m5.predict(X_test_final[k]),m6.predict(X_test_final[k]),m.predict(X_test_final[k])],axis=0)

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.

In [1488]:
print(accuracy_score(y_pred=y_preds.round(),y_true=y_test))

0.7023468948580976


In [1381]:
imp = importances(m,X_test_final[k],y_test)

In [1382]:
imp

,Importance
Feature,
enc_cab,0.1742
Type_of_Cab,0.0464
Trip_Distance,0.0252
Cancellation_Last_1Month,0.0164
Life_Style_Index,0.0156
Var3,0.0148
Confidence_Life_Style_Index,0.0136
Destination_Type,0.0096
Var1_isnull,0.0072


In [1603]:
df_test['Type_of_Cab'] = df_test['Type_of_Cab'].map(cab)
df_test['Confidence_Life_Style_Index'] = df_test['Confidence_Life_Style_Index'].map(conf)
df_test['Destination_Type'] = df_test['Destination_Type'].map(dest)
df_test['Gender'] = df_test['Gender'].map(gend)

In [1604]:
df_test['Type_of_Cab'].fillna(-999,inplace=True)
df_test['Var1_isnull'] = df_test['Var1'].apply(lambda x: 1 if np.isnan(x) == True else 0)
df_test['Var1'].fillna(df_train['Var1'].median(),inplace=True)
df_test['Confidence_Life_Style_Index'].fillna(-999,inplace=True)
df_test['Life_Style_Index'].fillna(-999,inplace=True)

In [1605]:
df_test['Customer_isnull'] =  df_test['Customer_Since_Months'].apply(lambda x: 1 if np.isnan(x) == True else 0)
df_test['Customer_Since_Months'].fillna(df_train['Customer_Since_Months'].median(),inplace=True)

In [1606]:
# df_test.drop('Confidence_Life_Style_Index',axis=1,inplace=True)
df_test['enc_cab'] = df_test['Type_of_Cab'].map(encoding)
# df_test['enc_dest'] = df_test['Destination_Type'].map(encoding)

In [1607]:
# df_test = pd.get_dummies(data=df_test,columns=['Confidence_Life_Style_Index','Destination_Type'])

In [1608]:
df_test.drop('Trip_ID',axis=1,inplace=True)
# df_test.drop('Type_of_Cab',axis=1,inplace=True)

In [1609]:
df_test[k].columns


Index(['enc_cab', 'Type_of_Cab', 'Trip_Distance', 'Cancellation_Last_1Month',
       'Confidence_Life_Style_Index', 'Destination_Type', 'Var3',
       'Customer_Rating', 'Life_Style_Index', 'Var1_isnull', 'Var1', 'Var2',
       'Customer_Since_Months', 'Customer_isnull'],
      dtype='object')

In [1610]:
X_train[k].columns

Index(['enc_cab', 'Type_of_Cab', 'Trip_Distance', 'Cancellation_Last_1Month',
       'Confidence_Life_Style_Index', 'Destination_Type', 'Var3',
       'Customer_Rating', 'Life_Style_Index', 'Var1_isnull', 'Var1', 'Var2',
       'Customer_Since_Months', 'Customer_isnull'],
      dtype='object')

In [1696]:
y_preds = np.mean([m1.predict(df_test[k]),m.predict(df_test[k])],axis=0)

In [1770]:
preds = m1.predict_proba(df_test[k]) * 0.12+ m.predict_proba(df_test[k]) * 0.86

In [1771]:
predictions = []

for x in preds:
    predictions.append(np.argmax(x))

In [1772]:
def get_vals(x):
    if x == 0:
        return 1
    elif x == 1:
        return 2
    else:
        return 3

In [1773]:
df_sub['Surge_Pricing_Type'] = predictions
df_sub['Surge_Pricing_Type'] = df_sub['Surge_Pricing_Type'].apply(lambda x:get_vals(x))
df_sub.to_csv('Final_Model.csv',index=False)

In [1094]:
    X_train['Dummy'] = np.random.randint(0,15)
    X_train['Dummy2'] = np.random.randint(0,100)
    X_train['Dummy3'] = np.random.uniform(0,1,size=X_train.shape[0])
    X_train['Dummy4'] = np.random.randint(0,1)

In [1095]:
from collections import defaultdict
from tqdm import tqdm
def _create_shadow(x):
    """
    Take all X variables, creating copies and randomly shuffling them
    :param x: the dataframe to create shadow features on
    :return: dataframe 2x width and the names of the shadows for removing later
    """
    x_shadow = x.copy()
    for c in x_shadow.columns:
        np.random.shuffle(x_shadow[c].values) # shuffle the values of each feature to all the features
    # rename the shadow
    shadow_names = ["shadow_feature_" + i for i in x.columns]
    x_shadow.columns = shadow_names
    # Combine to make one new dataframe
    x_new = pd.concat([x, x_shadow], axis=1)
    return x_new, shadow_names


# Set up the parameters for running the model in XGBoost
# params = {
#     'n_estimators':100,
#     'random_state':22,
#     'n_jobs':-1
# }
df = pd.DataFrame() # initial empty dataframe

n_iterations = 50
d = defaultdict(list)
for i in tqdm(range(1, n_iterations + 1)):
    # Create the shadow variables and run the model to obtain importances
# results = pd.DataFrame(columns='training_score', 'test_score')
    cv = StratifiedKFold(n_splits=3)
    score_avg = []
    for (train, test), i in zip(cv.split(X_train, y_train), range(5)):
        new_x, shadow_names = _create_shadow(X_train.iloc[train])
        valid_x,shadow_names = _create_shadow(X_train.iloc[test])
        model = lgb.LGBMClassifier(n_jobs=-1)
        model.fit(new_x,y_train.iloc[train])
        imp = importances(model,valid_x,y_train.iloc[test],n_samples=-1)
        imp.reset_index(inplace=True)
        if len(d) == 0:
            for i,j in imp.values:
                d[i].append(j)
        else:
            for i,j in imp.values:
                d[i].append(j)

100%|██████████| 50/50 [17:53<00:00, 21.47s/it]


In [1096]:
df_feature_importance = pd.DataFrame(data=d)
d_t = defaultdict(list)
for i in df_feature_importance.columns:
    mean_val = df_feature_importance[i].mean() * 100
    d_t[i].append(mean_val)
df_features = pd.DataFrame(data=d_t).T
df_features.reset_index(inplace=True)
df_features.sort_values(by=[0],ascending=False)

,index,0
0,enc_cab,14.949296
1,Type_of_Cab,11.797533
2,Trip_Distance,1.940432
3,Cancellation_Last_1Month,1.462713
4,Confidence_Life_Style_Index,1.198051
5,Destination_Type,0.994846
7,Var3,0.859824
6,Customer_Rating,0.832026
8,Life_Style_Index,0.792400
9,Var1_isnull,0.632314


In [1097]:
rf = df_features.sort_values(by=[0],ascending=False)
rf.reset_index(inplace=True,drop=True)

In [1365]:
k = rf[0:14]['index'].tolist()

In [1366]:
# k.remove('enc_cab')

In [1513]:
k

['enc_cab',
 'Type_of_Cab',
 'Trip_Distance',
 'Cancellation_Last_1Month',
 'Confidence_Life_Style_Index',
 'Destination_Type',
 'Var3',
 'Customer_Rating',
 'Life_Style_Index',
 'Var1_isnull',
 'Var1',
 'Var2',
 'Customer_Since_Months',
 'Customer_isnull']

In [1765]:
df_sub['Surge_Pricing_Type'].value_counts(normalize=True)

2    0.503862
3    0.340500
1    0.155638
Name: Surge_Pricing_Type, dtype: float64

In [1710]:
df_sub['Surge_Pricing_Type'].value_counts(normalize=True)

2    0.503724
3    0.340580
1    0.155695
Name: Surge_Pricing_Type, dtype: float64

In [1319]:
df_train['Surge_Pricing_Type'].value_counts(normalize=True)

2    0.430861
3    0.362443
1    0.206696
Name: Surge_Pricing_Type, dtype: float64

In [815]:
df_test.columns

Index(['Trip_ID', 'Trip_Distance', 'Type_of_Cab', 'Customer_Since_Months',
       'Life_Style_Index', 'Destination_Type', 'Customer_Rating',
       'Cancellation_Last_1Month', 'Var1', 'Var2', 'Var3', 'Gender',
       'mean_enc_Confidence_Life_Style_Index', 'Var1_isnull',
       'Customer_isnull'],
      dtype='object')

In [816]:
X_train.columns

Index(['Trip_Distance', 'Type_of_Cab', 'Customer_Since_Months',
       'Life_Style_Index', 'Destination_Type', 'Customer_Rating',
       'Cancellation_Last_1Month', 'Var1', 'Var2', 'Var3', 'Gender',
       'Var1_isnull', 'Customer_isnull',
       'mean_enc_Confidence_Life_Style_Index'],
      dtype='object')

In [ ]:
m  = lgb.LGBMClassifier(random_state=22,n_jobs=-1,colsample_bytree=1,min_child_samples=22,num_leaves=65,
                       lambda_l1=2.41,lambda_l2=5.21,learning_rate=0.1,n_estimators=100)